<a href="https://colab.research.google.com/github/jespimentel/jurimetria/blob/main/aulas_3_e_6_adapta%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importações necessárias
import os
import re
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
import openai

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_KEY')

In [1]:
# Cria uma sessão persistente
session = requests.Session()

# 1a requisição para obter os cookies
url_inicial = "https://esaj.tjsp.jus.br/cjpg/pesquisar.do"

# Dicionário de parâmetros da requisição
params = {
    "conversationId": "",
    "dadosConsulta.pesquisaLivre": "",
    "tipoNumero": "UNIFICADO",
    "numeroDigitoAnoUnificado": "",
    "foroNumeroUnificado": "",
    "dadosConsulta.nuProcesso": "",
    "dadosConsulta.nuProcessoAntigo": "",
    "classeTreeSelection.values": "",
    "classeTreeSelection.text": "",
    "assuntoTreeSelection.values": "",
    "assuntoTreeSelection.text": "",
    "agenteSelectedEntitiesList": "",
    "contadoragente": "0",
    "contadorMaioragente": "0",
    "cdAgente": "",
    "nmAgente": "",
    "dadosConsulta.dtInicio": "",
    "dadosConsulta.dtFim": "14/11/2024",
    "varasTreeSelection.values": "451-8",
    "varasTreeSelection.text": "1ª Vara Criminal",
    "dadosConsulta.ordenacao": "DESC",
}

response = session.get(url_inicial, params=params)

if response.status_code == 200:
    for i in range(1, 6):
        pagina = str(i)
        url_pagina = f"https://esaj.tjsp.jus.br/cjpg/trocarDePagina.do?pagina={pagina}&conversationId="

        # Usa a mesma sessão para fazer a requisição
        response = session.get(url_pagina)

        if response.status_code == 200:
            # Salva o conteúdo da página
            with open(f"resultado_pagina_{pagina}.html", "wb") as file:
                file.write(response.content)
            print(f"Página {pagina} salva com sucesso.")
        else:
            print(f"Erro ao acessar a página {pagina}: {response.status_code}")

        # Intervalo entre requisições
        time.sleep(2)
else:
    print("Erro ao acessar a página inicial.")

Página 1 salva com sucesso.
Página 2 salva com sucesso.
Página 3 salva com sucesso.
Página 4 salva com sucesso.
Página 5 salva com sucesso.


In [2]:
# Diretório contendo os arquivos HTML
directory = "/content/"

data = []

for filename in os.listdir(directory):
    if filename.endswith(".html"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r", encoding='utf-8') as f:  # Adicione encoding para lidar com caracteres especiais
            html_content = f.read()

        soup = BeautifulSoup(html_content, "html.parser")

        for result in soup.find_all("div", id="divDadosResultado"):
            for row in result.find_all("tr", class_="fundocinza1"):
                item = {}

                processo_link = row.find("a", title="Visualizar Inteiro Teor")
                if processo_link:
                    item["numero_do_processo"] = processo_link.find("span", class_="fonteNegrito").text.strip()
                else:
                    item["numero_do_processo"] = None

                table_data = row.find_all("td")
                if len(table_data) > 1:
                    info_table = table_data[1].find('table')
                    if info_table: # Verifica se a tabela de informações existe
                        for info_row in info_table.find_all("tr", class_="fonte"):
                            cells = info_row.find_all("td")
                            if cells:
                                if cells[0].strong:
                                    key = cells[0].strong.text.strip().replace(":", "")
                                    value = cells[0].text.replace(cells[0].strong.text,"").strip() if len(cells) == 1 else cells[1].text.strip()
                                    if key.lower() not in ["data de disponibilização"]:
                                        item[key.lower()] = value
                                    else:
                                        item['data_disponibilização'] = value
                            else:
                                pass  # Ou adicione um valor padrão se necessário

                else:
                    # Define valores padrão se não houver informações adicionais
                    item['classe'] = None
                    item['assunto'] = None
                    item['magistrado'] = None
                    item['comarca'] = None
                    item['foro'] = None
                    item['vara'] = None
                    item['data_disponibilização'] = None

                sentenca_divs = table_data[1].find_all("div", align="justify") if len(table_data) > 1 else [] # Lidar com casos onde table_data tem apenas um elemento
                full_text = ""
                for div in sentenca_divs:
                    text = div.get_text(strip=True)
                    full_text += text + '\n'
                item["sentença"] = full_text

                data.append(item)

df = pd.DataFrame(data)

# Ordenando as colunas
df = df[['numero_do_processo', 'classe', 'assunto', 'magistrado', 'comarca', 'foro', 'vara', 'data_disponibilização', 'sentença']]

df.sample()

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_disponibilização,sentença
24,1501194-07.2023.8.26.0599,Ação Penal - Procedimento Ordinário,Receptação,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,07/11/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...


# Teste com a API da OpenAI

In [3]:
!pip install --upgrade openai

In [4]:
perguntas = [
    "Informe os nomes completos dos réus. Se for mais de um, separe os nomes por ;",
    "Faça um breve resumo da decisão em um parágrafo de até 50 palavras",
    "A decisão foi procedente, improcedente ou parcialmente procedente? Responda apenas com uma palavra: procedente, parcialmente, improcedente",
    "Se a sentença foi procedente, qual a pena de prisão aplicada? Coloque a resposta no seguinte formato ano/meses/dias.\
    Exemplo: para uma pena de 5 anos e 4 meses, informe 5a/4m. Se for mais de um réu, coloque 10a/3m/11d (Alberto); 0a/2m/8d (João)",
]

novas_colunas = [
    "réus", "resumo", "merito", "penas"
]

# Chamada à API
def analisar_sentenca(sentenca):
    respostas = []
    for pergunta in perguntas:
        prompt = f"Sentença: {sentenca}\nPergunta: {pergunta}\nResposta:"

        try:
            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Você é um assistente jurídico que analisa decisões judiciais \
                    e responde de forma direta e objetiva. Siga o formato de resposta solicitado."},
                    {"role": "user", "content": prompt}
                ]
            )
            resposta = response.choices[0].message.content.strip()
        except Exception as e:
            resposta = f"Erro: {e}"

        respostas.append(resposta)
    return respostas

# Atualização do dataframe
for i, coluna in enumerate(novas_colunas):
    df[coluna] = df["sentença"].apply(lambda x: analisar_sentenca(x)[i])

In [5]:
display(df)

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_disponibilização,sentença,réus,resumo,merito,penas
0,1501782-14.2023.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,14/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,KAUÃ AIRTON SENA DOMINGUES,O réu KAUÃ AIRTON SENA DOMINGUES foi condenado...,Procedente,1a/8m.
1,1502210-93.2023.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,10/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,PAULO BUENO NETO,A sentença condenou Paulo Bueno Neto a um ano ...,Procedente,1a/8m.
2,3010500-31.2013.8.26.0451,Ação Penal - Procedimento Ordinário,Roubo Majorado,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,10/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,Matheus Valerio Dell Amatrice.,O juiz reconheceu a prescrição da pretensão ex...,Procedente,6a/2m/20d (Matheus Valerio Dell Amatrice)
3,1522521-98.2022.8.26.0451,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,10/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,GUILHERME SANTIN MARTINI.,O juiz desclassificou o crime imputado a Guilh...,improcedente,0a/0m.
4,0007099-94.2021.8.26.0451,Ação Penal - Procedimento Ordinário,Estelionato,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,08/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,FABIULA PAULA ROSA DE MEDEIROS.,Fabiula Paula Rosa de Medeiros foi condenada a...,procedente,1a/0m/0d (FABIULA PAULA ROSA DE MEDEIROS)
5,1501228-45.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,08/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,DOUGLAS JACQUIE,Douglas Jacquie foi condenado a cinco anos de ...,Procedente,5a/0m
6,0008548-63.2016.8.26.0451,Ação Penal - Procedimento Ordinário,Uso de documento falso,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,07/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,CLEBER AUGUSTO DE OLIVEIRA,O réu Cleber Augusto de Oliveira foi condenado...,parcialmente,2a/0m.
7,0014887-72.2015.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes contra o Meio Ambiente e o Patrimônio G...,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,07/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,Antonio Fernando Eduardo Matias,O juiz reconheceu a prescrição da pretensão pu...,Procedente,0a/0m/0d
8,1509440-87.2019.8.26.0451,Ação Penal - Procedimento Ordinário,Roubo,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,04/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,VINICIUS NOGUEIRA GITIRANA,O Juízo absolveu Vinicius Nogueira Gitirana da...,improcedente,"A sentença foi improcedente, portanto não houv..."
9,1501762-57.2022.8.26.0599,Ação Penal - Procedimento Ordinário,Furto,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,04/10/2024,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n...,JONAS ANDRE DOS SANTOS.,"O réu, JONAS ANDRE DOS SANTOS, foi condenado a...",Procedente,0a/4m/0d


In [6]:
# Salva o resultado em arquivo CSV
df.to_csv("processos.csv", index=False)